In [ ]:
!pip install boto3

In [2]:
import boto3
import tempfile
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NetflixAnalysisBoto3") \
    .master("local[*]") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.sql.execution.arrow.enabled", "true") \
    .getOrCreate()

In [3]:
spark.conf.get("spark.executor.cores")

'2'

In [6]:
# Carga del archivo CSV a Spark
netflix_titles = "netflix_titles.csv"
df_netflix=spark.read.csv(netflix_titles,header=True,inferSchema=True)

In [ ]:
df_netflix.show()

In [8]:
df_netflix.printSchema()

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- cast: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)
 |-- description: string (nullable = true)



In [9]:
from pyspark.sql.functions import corr, col,count,when

In [12]:
df_netflix.groupBy("type").count().show()

+-------------+-----+
|         type|count|
+-------------+-----+
|         NULL|    1|
|      TV Show| 2676|
|        Movie| 6131|
|William Wyler|    1|
+-------------+-----+



In [13]:
df_netflix.groupBy("type","director").count().show()

+-------+--------------------+-----+
|   type|            director|count|
+-------+--------------------+-----+
|  Movie|    Theodore Witcher|    1|
|  Movie|         Paul Haggis|    2|
|  Movie|     Dado C. Lumibao|    1|
|  Movie|     Vishal Mahadkar|    1|
|  Movie|   Jonathan Augustin|    1|
|  Movie|        Ahmed Medhat|    1|
|  Movie|  Naman Nitin Mukesh|    1|
|TV Show|     Quek Shio-chuan|    1|
|  Movie|            Muh Chen|    1|
|  Movie|         David Mrnka|    1|
|  Movie|       Peter Sattler|    1|
|  Movie|      Richard Brooks|    1|
|  Movie|        Tony Leondis|    1|
|  Movie|         Sam O'Steen|    1|
|  Movie|    Steven C. Miller|    3|
|  Movie|         Tolga Örnek|    1|
|  Movie|            Rana Eid|    1|
|  Movie|            Sam Rega|    1|
|  Movie|  Anand Ravichandran|    1|
|  Movie|Tamas Yvan Topola...|    1|
+-------+--------------------+-----+
only showing top 20 rows



In [19]:
df_netflix.groupBy("rating").count().show()

+--------------------+-----+
|              rating|count|
+--------------------+-----+
|    November 1, 2020|    1|
|    Shavidee Trotter|    1|
|       Adriane Lenox|    1|
|                TV-Y|  307|
|       Maury Chaykin|    1|
|                2019|    1|
|                2017|    1|
|                  UR|    3|
| Keppy Ekpenyong ...|    1|
|      Benn Northover|    1|
|Classic Movies, D...|    1|
|                NULL|    6|
|                  PG|  286|
|         Jide Kosoko|    1|
|               TV-MA| 3195|
|     Jowharah Jones"|    1|
|            TV-Y7-FV|    6|
|                2006|    1|
|      Itziar Aizpuru|    1|
|              84 min|    1|
+--------------------+-----+
only showing top 20 rows



In [20]:
df_netflix.groupBy("release_year").count().show()

+-----------------+-----+
|     release_year|count|
+-----------------+-----+
|     Ted Ferguson|    1|
|             1987|    8|
|             1956|    2|
|             2016|  901|
|             2020|  952|
|             2012|  237|
|             1958|    3|
|           40 min|    1|
|             1943|    3|
|             1972|    5|
| Marquell Manning|    1|
|             1988|   18|
|             2019| 1026|
|             2017| 1030|
|             1977|    7|
|             2014|  352|
|             1971|    5|
|             1984|   12|
|             2013|  288|
|             1982|   17|
+-----------------+-----+
only showing top 20 rows



In [21]:
df_netflix.groupBy("date_added").count().show()

+------------------+-----+
|        date_added|count|
+------------------+-----+
|      May 21, 2021|    8|
|     March 2, 2021|    3|
|September 23, 2020|    2|
| September 8, 2020|    4|
|    April 14, 2020|    2|
| December 30, 2019|    1|
|   August 12, 2019|    1|
|     June 22, 2019|    2|
|      May 30, 2017|    2|
|    April 29, 2016|    2|
|    March 25, 2016|    1|
|  October 27, 2015|    1|
|   January 1, 2008|    1|
|     March 2, 2017|    2|
|  October 31, 2015|    1|
|     June 23, 2021|    5|
|  November 1, 2020|   32|
|  February 9, 2020|    3|
| November 28, 2019|    6|
|   October 5, 2019|    3|
+------------------+-----+
only showing top 20 rows

